<a href="https://colab.research.google.com/github/mariosilvestri3/capstone-data/blob/main/data-preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TelCo Customer Churn - Capstone for Mario Silvestri
## Objective 2 - Data Preparation

### Load Dataset from Datasource

In [1]:
import pandas as pd

churn_data_file = 'https://raw.githubusercontent.com/mariosilvestri3/capstone/bf1edf1b098c856b49eaf32e6832c91ff891bd74/data/customer-churn-raw.csv'
df = pd.read_csv(churn_data_file)

### Examine and Understand Dataset

In [2]:
df.head()

,LoyaltyID,Customer ID,Senior Citizen,Partner,Dependents,Tenure,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn
0,318537,7590-VHVEG,No,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,152148,5575-GNVDE,No,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,326527,3668-QPYBK,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,845894,7795-CFOCW,No,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,503388,9237-HQITU,No,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
df.tail()

,LoyaltyID,Customer ID,Senior Citizen,Partner,Dependents,Tenure,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn
7038,810338,6840-RESVB,No,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,230811,2234-XADUH,No,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,155157,4801-JZAZL,No,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,731782,8361-LTMKD,Yes,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes
7042,353947,3186-AJIEK,No,No,No,66,Yes,No,Fiber optic,Yes,No,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),105.65,6844.5,No


In [4]:
df.columns

Index(['LoyaltyID', 'Customer ID', 'Senior Citizen', 'Partner', 'Dependents',
       'Tenure', 'Phone Service', 'Multiple Lines', 'Internet Service',
       'Online Security', 'Online Backup', 'Device Protection', 'Tech Support',
       'Streaming TV', 'Streaming Movies', 'Contract', 'Paperless Billing',
       'Payment Method', 'Monthly Charges', 'Total Charges', 'Churn'],
      dtype='object')

In [5]:
df.shape

(7043, 21)

#### Dataset has 7,043 rows and 21 columns. Churn column is our dependant variable. 

#### LoyaltyID and Customer ID both appear to be unique customer identifiers. Let's verify.

In [7]:
df.iloc[:, 0:2].nunique()

LoyaltyID      7021
Customer ID    7043
dtype: int64

#### LoyaltyID is not a unique identifier because it is repeated. Consulting with a subject matter expert on this datasource would provide more information as to why some customers share a Loyalty ID.



### Describe Data
|Attribute|Statistical Type|
|---|---|
|LoyaltyID| Non-unique Identifier|
|Customer ID|Unique Identifier
|Senior Citizen|Nominal
|Partner|Nominal
|Tenure|Ratio|
|Phone Service|Nominal|
|Multiple Lines|Nominal|
|Internet Service|Nominal|
|Online Security|Nominal|
|Online Backup|Nominal|
|Device Protection|Nominal|
|Tech Support|Nominal|
|Streaming TV|Nominal|
|Streaming Movies|Nominal|
|Contract|Nominal|
|Paperless Billing|Nominal|
|Payment Method|Nominal|
|Monthly Charges|Ratio|
|Total Charges|Ratio|
|Churn|Nominal|

### Verify Data Quality

### Analyze Data